# AI Pair Programming

Dans ce notebook nous allons demander à une intelligence artificielle de générer du code Python pour réaliser des tâches de traitement automatique de corpus.

Avant de commencer, choisissez un outil comme [Bard](https://bard.google.com/u/2/chat) ou [ChatGPT](https://chat.openai.com/) et créez un compte.

Vous pouvez ensuite demander à l'outil de créer du code. Avant de commencer, n'hésitez pas à lire [cet article](https://exocoding.com/ai-code-generation/) qui détaille les **bonnes pratiques** pour créer des _prompts_ efficaces dans le cadre de la génération de code par l'intelligence artificielle.


### 1. Algorithme simple en Python

Demandez à l'IA de générer un code python qui lance un décompte du réveillon du Nouvel An. Le code doit imprimer les nombres de 10 à 0 avec un intervalle d'une seconde, puis imprimer "Bonne année" à la fin.

In [1]:
# Votre code ici
import time

for i in range(10, -1, -1):
    print(i)
    if i > 0:
        time.sleep(1)

print("Bonne année !")

10
9
8
7
6
5
4
3
2
1
0
Bonne année !


### 2. Détection du sujet de la phrase

Demandez à l'IA d'extraire le sujet dans une phrase.
Demandez ensuite de générer le code Python qui réalise cette tâche et testez le ci-dessous.

In [2]:
# Votre code ici
def extraire_sujet(phrase: str) -> str:
    # On suppose que le verbe est "est" et on prend tout ce qui est avant
    separateur = " est"
    if separateur in phrase:
        sujet = phrase.split(separateur)[0].strip()
        return sujet
    else:
        # Si "est" n'est pas trouvé, on peut retourner la phrase entière
        return phrase.strip()

# Exemple d'utilisation
phrase = "La Réunion est une très belle île"
sujet = extraire_sujet(phrase)
print("Sujet extrait :", sujet)


Sujet extrait : La Réunion


### 3. Entités nommmées liées à Wikidata

Demandez à l'IA d'extraire les entités nommées d'un texte en français, et de les lier à un identifiant wikidata.
Demandez ensuite à l'IA de générer le code Python pour réaliser cette tâche et testez le ci-dessous.

In [9]:
# Votre code ici
import spacy

# 1. Installer le modèle une fois dans le terminal :
#    python -m spacy download fr_core_news_sm

# 2. Puis lancer ce script
nlp = spacy.load("fr_core_news_md")

texte = ("La dernière version du plan de paix sur l’Ukraine est jugée "
         "« significativement meilleure » pour Kiev, permettant notamment "
         "à l’Ukraine de conserver une armée de 800.000 hommes.")

doc = nlp(texte)

print("Entités nommées détectées :")
for ent in doc.ents:
    print(ent.text, "->", ent.label_)

import spacy
from spacyfishing import EntityFishing

# Charger spaCy français
nlp = spacy.load("fr_core_news_md")

# Ajouter le composant entity-fishing au pipeline
nlp.add_pipe("entityfishing", config={
    "url": "https://cloud.science-miner.com/nerd"  # ou ton serveur entity-fishing
})

texte = ("La dernière version du plan de paix sur l’Ukraine est jugée "
         "« significativement meilleure » pour Kiev, permettant notamment "
         "à l’Ukraine de conserver une armée de 800.000 hommes.")

doc = nlp(texte)

for ent in doc.ents:
    # ent._.kb_qid contient normalement l’ID Wikidata (ex: 'Q212' pour l’Ukraine)
    print(ent.text, "->", ent._.kb_qid)

Entités nommées détectées :
Ukraine -> LOC
Kiev -> LOC
Ukraine -> LOC


ModuleNotFoundError: No module named 'spacyfishing'

### 4. A vous de jouer

Pensez à une analyse que vous voudriez faire sur un texte. Demandez à l'IA de générer un code python qui réalise cette analyse et testez le ci-dessous

In [10]:
# Votre code ici
from transformers import pipeline

# 1. Pipeline de reconnaissance d'entités nommées (NER)
ner_pipe = pipeline(
    "ner",
    model="Davlan/xlm-roberta-base-ner-hrl",   # modèle NER multilingue
    aggregation_strategy="simple"             # regroupe les tokens d'une même entité
)

# 2. Pipeline d'analyse des sentiments (multi-langues)
sentiment_pipe = pipeline(
    "sentiment-analysis",
    model="tabularisai/multilingual-sentiment-analysis"
)

texte = (
    "C’est toujours le même cirque. Chaque annonce d’un nouvel album de Kanye West "
    "charrie son lot de controverses, d’indignations, d’outrances et de sorties "
    "médiatiques fracassantes. Et tout le monde de tomber à chaque fois dans le "
    "panneau : Kanye West, artiste total ou escroc ? Génie ou demeuré ? Prophète "
    "ou pervers narcissique ?"
)

# 1) Extraction des entités nommées
ents = ner_pipe(texte)

# 2) Pour chaque entité, on analyse le sentiment du texte qui la contient
resultats = []
for ent in ents:
    span_text = ent["word"]
    # on passe toute la phrase au modèle de sentiments (ici : le texte complet pour simplifier)
    sentiment = sentiment_pipe(texte)[0]
    resultats.append({
        "entite": span_text,
        "type": ent["entity_group"],
        "sentiment_label": sentiment["label"],
        "sentiment_score": float(sentiment["score"])
    })

# Affichage des résultats
for r in resultats:
    print(
        f"Entité: {r['entite']:<25} "
        f"Type: {r['type']:<8} "
        f"Sentiment: {r['sentiment_label']:<10} "
        f"Score: {r['sentiment_score']:.3f}"
    )


c:\Users\durie\Documents\tac\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\durie\Documents\tac\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\durie\.cache\huggingface\hub\models--Davlan--xlm-roberta-base-ner-hrl. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activ

NameError: name 'torch' is not defined


### Pour aller plus loin...

En tant qu'étudiant ULB vous avez accès gratuitement au [Github student pack](https://education.github.com/pack). Ce pack vous donne permet d'utiliser [Github copilot](https://github.com/features/copilot), un outil d'auto-complétion de code grâce à l'intelligence artificielle. Ceci peut être très utile si vous voulez réaliser des tâches complexes sans être un expert en python.